STEP 1: Import OpenStack data from Opendev. The output is a list of JSON files

In [ ]:
import pandas as pd
import numpy as np
import json
import requests
from pathlib import Path
import pandas as pd
from itertools import count

def GetData(skip): 
    params = {
        'O': 97,
        'n': skip,
        #'S': skip
        'S': skip*500
        }
    response = requests.get("https://review.opendev.org/changes/?q=status:merged repositories:openstack&o=CURRENT_FILES&o=MESSAGES&o=CURRENT_COMMIT&o=CURRENT_REVISION", params=params)
    
    data_per_request = response.text.split("\n")[1]
    data_per_request = np.array(json.loads(data_per_request))
    
    if len(data_per_request) > 0: 

       skip += 1 
       filename = 'data_%d.json'%(skip,)
       pd.Series(data_per_request).to_json(filename, orient='records') 
       GetData(skip)  
    
if __name__ == '__main__':   
   skip = 0
   GetData(skip) 
   

STEP 2: Merge all JSON files into one

In [5]:
import os
files =[]
for file in os.listdir("/content"):
    if file.endswith(".json"):
        files.append(os.path.join("/content", file))
        #print(os.path.join("/content", file))

def merge_JsonFiles(filename):
    result = list()
    for f1 in filename:
        with open(f1, 'r') as infile:
            result.extend(json.load(infile))

    with open('/content/FINALJSON.json', 'w') as output_file:
        json.dump(result, output_file)

merge_JsonFiles(files)

In [ ]:


with open('/content/FINALJSON.json','r') as string:
    my_dict=json.load(string)
    #print(my_dict[5])
string.close()

for x in my_dict:
		keys = x.keys()
		print(keys)
		values = x.values()
		print(values)


In [ ]:
df = pd.DataFrame(my_dict)
#Save the data in a CSV file

filepath = Path('./openstack_data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath, index=False)

df

##Put messages in single csv for each code review

In [ ]:
pd.Series(df['messages']).to_json("./messages.json", orient='records')

In [ ]:
Msg_data=[]
with open('./messages.json', 'r') as f:
    distros_dict = json.load(f)
for distro in distros_dict:
    #print(distro)
    # we do not consider CC reviewers as are bots
    #print(distro['CC'])
    for x in distro:
		    #print(x.keys(), x.values())
	      #print(list(x.keys()))
        #print(list(x.values()))
			  Msg_data.append(list(x.values()))
Msg_data		 

In [ ]:

dfM = pd.DataFrame(Msg_data)
#Save the data in a CSV file
dfM.columns = ['id', 'tag', 'author', 'real_author', 'date', 'message', 'accounts_in_message', '_revision_number']
filepath = Path('./openstack_data_Messages.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
dfM.to_csv(filepath, index=False)

dfM

## Get the reviewers count for each code review with their usernames

In [ ]:
print(df['reviewers'])

In [ ]:
pd.Series(df['reviewers']).to_json("./Reviewers.json", orient='records')

In [ ]:
Rev_data=[]
with open('./Reviewers.json','r') as string:
    Rev_dict=json.load(string)
    print(Rev_dict[5])
string.close()

for x in Rev_dict:
		#keys = x.keys()
		#print(keys)
		#values = x.values()
		#print(values)
    Rev_data.append(list(x.values()))
Rev_data

In [ ]:
## Extract number of reviewers 
Nb_Reviewers=[]
nb=0
flat_list = [item for sublist in Rev_data for item in sublist]
for r in flat_list:
  #print(r)
  nb=0
  for r1 in r:
    #print(r1)
    #print(list(r1.keys()))
    count_reviewer = list(r1.keys()).count('_account_id')
    nb=nb+count_reviewer
  Nb_Reviewers.append(nb)
Nb_Reviewers 

In [ ]:
## Extract names of reviewers 
list_intermediare=[]
Names_Reviewers=[]
flat_list = [item for sublist in Rev_data for item in sublist]
for r in flat_list:
  #print(r)
  
  for r1 in r:
    #print(r1)
    #print(list(r1.keys()))
    #print(list(r1.values())[3])
    name= list(r1.values())[1]
    print(name)
    list_intermediare.append(name)
  print(list_intermediare)
  Names_Reviewers.append(list_intermediare) 
  print(Names_Reviewers)
  list_intermediare.clear()
  print(list_intermediare)
  
#Names_Reviewers 

In [ ]:
Rev_data=[]
with open('./Reviewers.json', 'r') as f:
    distros_dict = json.load(f)
for distro in distros_dict:
    #print(distro['REVIEWER'])
    # we do not consider CC reviewers as are bots
    #print(distro['CC'])
    Rev_data.append(distro['REVIEWER'])
    for x in distro['REVIEWER']:
		  
		    print(x.keys(), x.values())
		
		

In [ ]:

df = pd.DataFrame(distro['REVIEWER'])
#Save the data in a CSV file

filepath = Path('./openstack_data_reviewers.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath, index=False)

df